In [10]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [11]:
# Configuração dos parâmetros do banco
host='localhost'
port=3306
database='constructal_automate_results'
user='constructal_automate'
password='constructal_design_2024'

query = f'''
  WITH RankedValues AS (
    SELECT sp.N_ls, sp.N_ts, sp.k, sp.h_s, sp.t_s, epb.sigma_u, epb.w_max, 
           ROUND(epb.sigma_u / 59.68, 3) AS NUS_2A, 
           ROUND(epb.w_max / 38.0533, 3) AS NMD_2A,
           ROW_NUMBER() OVER (PARTITION BY sp.N_ls, sp.N_ts ORDER BY ROUND(epb.w_max / 38.0533, 3) ASC) AS RowNum
    FROM constructal_automate_results.cbeb_elastoplasticbuckling epb
    INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis spa 
        ON epb.stiffened_plate_analysis_id = spa.id
    INNER JOIN constructal_automate_results.csg_stiffenedplate sp 
        ON spa.stiffened_plate_id = sp.id
    WHERE spa.buckling_load_type_id = 2 
      AND epb.id NOT BETWEEN 259 AND 266
      AND sp.t_s IN (10, 15, 20)
)
SELECT N_ls, N_ts, k, h_s, t_s, sigma_u, w_max, NUS_2A, NMD_2A
FROM RankedValues
WHERE RowNum = 1;
'''

base_path = 'tabelas_dissertacao/'

In [12]:
file_name = 'estudo_caso_melhores_desempenhos_nmd_gdl'
csv_name = file_name + ".csv"
latex_name = file_name + ".tex"

In [13]:
def retrieve_data_from_db(host, port, user, password, database, query):
    try:
        # Criar a conexão com o banco de dados
        connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_string)

        # Executar a query e carregar os dados em um DataFrame
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Erro: {e}")

In [14]:
def general_formatter(fmt, value):
    return f'{fmt}' % value

def formatter_d(value):
    return '%d' % value

def formatter_float1(value):
    return '%.1f' % value

def formatter_float2(value):
    return '%.2f' % value

def formatter_float3(value):
    return '%.3f' % value

def formatter_float4(value):
    return '%.4f' % value

In [15]:
def generate_latex_from_df(df):
    # Salvar o resultado como um arquivo CSV
    df.to_csv(base_path + csv_name, index=False)

    formatters = ['%d', '%d']

    # Converter o DataFrame para LaTeX
    print(f"Convertendo o arquivo CSV para LaTeX")
    latex_table = df.to_latex(formatters=[formatter_d, formatter_d, formatter_float3, formatter_d, formatter_d, formatter_float2, formatter_float4, formatter_float3, formatter_float3],
                              decimal=',',
                              index=False)

    # Salvar a tabela LaTeX em um arquivo .tex
    with open(base_path + latex_name, "w") as f:
        f.write(latex_table)

    print("Tabela LaTeX criada com sucesso!")

In [16]:
df_results = retrieve_data_from_db(host, port, user, password, database, query)

In [17]:
generate_latex_from_df(df_results)

Convertendo o arquivo CSV para LaTeX
Tabela LaTeX criada com sucesso!
